## Segmenting Neighborhoods in Toronto

#### Section 0: Required packages and General Info

This notebook requires numpy, pandas, lxml, json (the python module), geopy, matplotlib, and folium to run.

For the clustering and segmenting Toronto assignment reviewers:
Question 1 is lines 1-11. 
Question 2 is lines 12-15. 
Question 3 is lines 16-the end.

#### Section 1: Scraping Neighborhoods from Toronto Postal Codes

##### Step 1: importing required packages

In [1]:
import requests
import lxml.html as lh
import pandas as pd
#import beautifulsoup4 as soup  I'm having some environment issues while trying to import beautifulsoup4. Not used for analysis.

##### Step 2: Store the website we're scraping as a variable, and store the elements from the table

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
page = requests.get(url)
doc = lh.fromstring(page.content)
table_elements = doc.xpath('//tr')

In [4]:
[len(T) for T in table_elements[:12]] #verifies I pulled elements from just the table I wanted

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0 #For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighborhood
"


Now onto creating a dictionary of the postal codes, boroughs, and neighorhoods. When all is said and done for the dataframe, we want one row for each postal code, which corresponds to one or more neighborhoods within one borough. If a postal code is "Not assigned" in the borough cell, it is skipped over.

In [6]:
postaldict = {}

j = 0
for r in range(1,len(tr_elements)):
    j+=1
    row=tr_elements[r]
    
    if len(row)!=3:
        break
    item = row.text_content().split('\n')

    if item[2] != 'Not assigned':
        if item[1] not in postaldict.keys():
            postaldict[item[1]] = [item[2], [item[3]]]
        else:
            postaldict[item[1]][1].append(item[3])

##### Step 3: Clean-up

Assign any "Not assigned" neighborhoods the name of the borough, and convert the neighborhood lists into strings

In [7]:
for key in postaldict:
    if 'Not assigned' in postaldict[key][1]:
        postaldict[key][1] = postaldict[key][0]
    elif type(postaldict[key][1]) != str:
        y = ", ".join(postaldict[key][1])
        postaldict[key][1] = y
        #print(postaldict[key][1])
print(postaldict)

{'M3A': ['North York', 'Parkwoods'], 'M4A': ['North York', 'Victoria Village'], 'M5A': ['Downtown Toronto', 'Harbourfront'], 'M6A': ['North York', 'Lawrence Heights, Lawrence Manor'], 'M7A': ["Queen's Park", "Queen's Park"], 'M9A': ['Downtown Toronto', "Queen's Park"], 'M1B': ['Scarborough', 'Rouge, Malvern'], 'M3B': ['North York', 'Don Mills North'], 'M4B': ['East York', 'Woodbine Gardens, Parkview Hill'], 'M5B': ['Downtown Toronto', 'Ryerson, Garden District'], 'M6B': ['North York', 'Glencairn'], 'M9B': ['Etobicoke', 'Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park'], 'M1C': ['Scarborough', 'Highland Creek, Rouge Hill, Port Union'], 'M3C': ['North York', 'Flemingdon Park, Don Mills South'], 'M4C': ['East York', 'Woodbine Heights'], 'M5C': ['Downtown Toronto', 'St. James Town'], 'M6C': ['York', 'Humewood-Cedarvale'], 'M9C': ['Etobicoke', 'Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe'], 'M1E': ['Scarborough', 'Guildwood, Morningside, West Hill'],

For the dataframe, we want the index to be numeric. With that in mind, I'm making a dictionary with a number as the index, and the postal code as the frist element in the value.

In [8]:
finaldict = {}
i=0
for k in postaldict.keys():
    finaldict[i] = [k,postaldict[k][0],postaldict[k][1]]
    i+=1
len(finaldict)

103

##### Final Step: Creating the Dataframe from the dictionary

In [9]:
df = pd.DataFrame.from_dict(finaldict,orient='index', columns = ['PostalCode','Borough', 'Neighborhood'])

In [10]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
df.shape

(103, 3)

#### Section 2: Retrieving the geographic coordinates of each postal code

In [ ]:
!conda install -c conda-forge geopy --yes

In [17]:
from geopy.geocoders import Nominatim

ModuleNotFoundError: No module named 'geocoder'

In [12]:
geocodecsv = "C:/Users/Mercedes/Documents/DSCapstone/Coursera_Capstone/Geospatial_Coordinates.csv"
geodf = pd.read_csv(geocodecsv)
geodf.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
geodf.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


The numeric indexes of the pre-existing dataframe and the coordinate dataframe do not line up, so we'll be joining them together, using PostalCode as the unifying column. 

In [13]:
fulldf = pd.merge(df,geodf, how = 'outer', on = 'PostalCode', left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

In [14]:
fulldf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [15]:
fulldf.shape

(103, 5)

#### Section 3: Exploring the Neighborhoods in Toronto Using Foursquare

In [16]:
import numpy as np

import json
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [17]:
import folium

In [18]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(fulldf['Latitude'], fulldf['Longitude'], fulldf['Borough'], fulldf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#229c43',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's narrow the scope to neighborhoods with 'Toronto' as the borough name.

In [19]:
d_toronto_data = fulldf[fulldf['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
d_toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
9,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576


In [20]:
map_dtoronto = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# add markers to map
for lat, lng, label in zip(d_toronto_data['Latitude'], d_toronto_data['Longitude'], d_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#f77a3b',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dtoronto)  
    
map_dtoronto

In [21]:
CLIENT_ID = 'LU44KGPLDS0NUWMCFLAVERAHBCFONLR33RZD4A3DJPQRQDV5' # your Foursquare ID
CLIENT_SECRET = 'MHJEV2SWS0CG5KIJLY0GUK2Z2OTOL5BK2LRQFVQEXISZBLKC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LU44KGPLDS0NUWMCFLAVERAHBCFONLR33RZD4A3DJPQRQDV5
CLIENT_SECRET:MHJEV2SWS0CG5KIJLY0GUK2Z2OTOL5BK2LRQFVQEXISZBLKC


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
d_toronto_venues = getNearbyVenues(names=d_toronto_data['Neighborhood'],
                                   latitudes=d_toronto_data['Latitude'],
                                   longitudes=d_toronto_data['Longitude']
                                  )


Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


In [24]:
print(d_toronto_venues.shape)
d_toronto_venues.head()
d_toronto_venues.groupby('Neighborhood').count()

(1282, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",91,91,91,91,91,91
Christie,17,17,17,17,17,17
Church and Wellesley,86,86,86,86,86,86
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [25]:
print('There are {} unique categories.'.format(len(d_toronto_venues['Venue Category'].unique())))

There are 205 unique categories.


In [26]:
# one hot encoding
d_toronto_onehot = pd.get_dummies(d_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to datafra
d_toronto_onehot['Neighborhood'] = d_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [d_toronto_onehot.columns[-1]] + list(d_toronto_onehot.columns[:-1])
d_toronto_onehot = d_toronto_onehot[fixed_columns]

d_toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
d_toronto_onehot.shape

(1281, 206)

In [27]:
d_toronto_grouped = d_toronto_onehot.groupby('Neighborhood').mean().reset_index()
d_toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.020000,0.000000,...,0.010000,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.017241,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.021739,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.011765,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011765,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.011765,0.000000,0.000000,0.011765,0.000000
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.054945,0.000000,0.054945,0.010989,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.023256,0.011628,0.000000,0.000000,0.000000,0.0,0.000000,0.011628,0.000000,...,0.011628,0.011628,0.00,0.00,0.00,0.000000,0.000000,0.011628,0.000000,0.011628
8,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000


In [28]:
d_toronto_grouped.shape

(18, 205)

In [29]:
num_top_venues = 5

In [31]:
for hood in d_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = d_toronto_grouped[d_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2     Cheese Shop  0.03
3  Farmers Market  0.03
4        Beer Bar  0.03


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2  Airport Terminal  0.13
3     Boat or Ferry  0.07
4   Harbor / Marina  0.07


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.07
1   Restaurant  0.04
2       Market  0.04
3     Pharmacy  0.04
4  Pizza Place  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.05
2  Italian Restaurant  0.05
3        Burger Joint  0.04
4      Ice Cream Shop  0.04


----Chinatown, 

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = d_toronto_grouped['Neighborhood']

for ind in np.arange(d_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(d_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,French Restaurant,Farmers Market
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Boat or Ferry
3,"Cabbagetown, St. James Town",Coffee Shop,Market,Italian Restaurant,Pub,Pizza Place
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint


In [34]:
# set number of clusters
kclusters = 5

d_toronto_grouped_clustering = d_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(d_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 4, 0, 0, 0])

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

d_toronto_merged = d_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
d_toronto_merged = d_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

d_toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Bakery,Park,Pub,Mexican Restaurant
1,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,Café,Restaurant,Coffee Shop,Hotel,Bakery
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.0,Coffee Shop,Cocktail Bar,Café,French Restaurant,Farmers Market
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0.0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4.0,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,0.0,Coffee Shop,Café,Bar,Steakhouse,Bakery
8,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0.0,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout
9,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0.0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant


I am fairly certain the oordinates for Queen's Park are erroneous, and covered by the Queen's Park borough, so I am dropping the Queen's Park column with the NaNs.

In [43]:
d_toronto_merged.dropna(axis=0, inplace=True)
d_toronto_merged = d_toronto_merged.reset_index(drop = True)
d_toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Bakery,Park,Pub,Mexican Restaurant
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,Café,Restaurant,Coffee Shop,Hotel,Bakery
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.0,Coffee Shop,Cocktail Bar,Café,French Restaurant,Farmers Market
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0.0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint


The 'Cluster Labels' column values were inserted as floats, but they need to be integers to be iterable. I will now make that change.

In [60]:
d_toronto_merged['Cluster Labels'] = d_toronto_merged['Cluster Labels'].astype('int64')
d_toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Mexican Restaurant
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Restaurant,Coffee Shop,Hotel,Bakery
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Café,French Restaurant,Farmers Market
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports
6,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,0,Coffee Shop,Café,Bar,Steakhouse,Bakery
7,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout
8,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Café,Hotel,Restaurant,Gym


In [59]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(d_toronto_merged['Latitude'], d_toronto_merged['Longitude'], d_toronto_merged['Neighborhood'], d_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The neighborhoods with coffee shops and cafes in the top 3 venues were clustered together. 